# Algoritmos de optimización - Reto 3

Nombre: Julio Emanuel Suriano Bryk

Link en Github: https://github.com/EmaSuriano/python-demos/blob/main/submits/Algoritmos_R3.ipynb

Link en Google Colab: https://colab.research.google.com/github/EmaSuriano/python-demos/blob/main/submits/Algoritmos_R3.ipynb


Implementar uno de los siguientes retos:

- Mejorar la implementación de búsqueda local implementada en clase sobre el TSP con otros operadores de vecindad.
- Implementar el algoritmo de búsqueda tabú para el TSP de la AG3.
- Mejorar la implementación de recocido simulado implementado en clase sobre el TSP eligiendo una generación de solución vecina con un grado de aleatoriedad menor (función genera_vecino_aleatorio()).
- Mejorar la implementación de colonia de hormigas implementada en clase sobre el TSP mediante una elección de nodo que tenga en consideración una función de probabilidad que depende de las feromonas.
- **Mejorar la implementación del algoritmo genético propuesto para la resolución del TSP.**

El formato de entrega será el de Jupyter Notebook exportado a HTML. Se debe mostrar el resultado de ejecución de las celdas. Se debe mostrar el resultado de, al menos, dos pruebas de cada algoritmo implementado para condiciones de entrada diferentes. El código debe estar debidamene comentado. Se deberá adjuntar también el archivo con extensión IPYNB o un enlace a Github por si es necesaria la ejecución de código. Los distintos ficheros se comprimirán en formato _.ZIP o _.RAR para su entrega en el aula virtual.


### Mejoras hechas al algoritmo genético

1. Utilización de la cruza uniforme (`uniform_cross`) para generar nuevos genes.
2. Valores de Mutación y Elitismo adaptativo dependiendo de las iteración: Empezando con valores de mutación alto y elitismo bajo (Diversificación), y a medida que iteramos en las generaciones el elitismo aumenta y la mutación decrece (Intensificación).
3. Implementación de algoritmo de mutación utilizando método de intercambiar la posición de un elemento.


In [1]:
#Modulo de llamadas http para descargar ficheros
%pip install requests

#Libreria del problema TSP: http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsplib.html
%pip install tsplib95

In [19]:
import tsplib95
from urllib import request
import gzip
import random
from typing import Dict
import math
import numpy as np
import copy  # Permite hacer copias de objetos en python: listas, diccionarios,...

In [3]:
# basic typehint for problem
Problem = Dict


def load_problem(file="swiss42.tsp") -> Problem:
    """load problem from remote url

    Returns:
        problem: problem with the configuration of nodes done
    """
    zip_file = f"{file}.gz"

    request.urlretrieve(
        f"http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/{zip_file}",
        zip_file,
    )

    open(file, "wb").write(gzip.open(zip_file, "rb").read())

    return tsplib95.load(file)


problem = load_problem()

print(problem.render())

NAME: swiss42
COMMENT: 42 Staedte Schweiz (Fricker)
TYPE: TSP
DIMENSION: 42
EDGE_WEIGHT_TYPE: EXPLICIT
EDGE_WEIGHT_FORMAT: FULL_MATRIX
EDGE_WEIGHT_SECTION:
0 15 30 23 32 55 33 37 92 114 92 110 96 90 74 76 82 67 72 78 82 159 122 131 206 112 57 28 43 70 65 66 37 103 84 125 129 72 126 141 183 124
15 0 34 23 27 40 19 32 93 117 88 100 87 75 63 67 71 69 62 63 96 164 132 131 212 106 44 33 51 77 75 72 52 118 99 132 132 67 139 148 186 122
30 34 0 11 18 57 36 65 62 84 64 89 76 93 95 100 104 98 57 88 99 130 100 101 179 86 51 4 18 43 45 95 45 115 93 152 159 100 112 114 153 94
23 23 11 0 11 48 26 54 70 94 69 89 75 84 84 89 92 89 54 78 99 141 111 109 190 89 44 11 29 54 56 89 47 118 96 147 151 90 122 126 163 101
32 27 18 11 0 40 20 58 67 92 61 78 65 76 83 89 91 95 43 72 110 141 116 105 190 81 34 19 35 57 63 97 58 129 107 156 158 92 129 127 161 95
55 40 57 48 40 0 23 55 96 123 78 75 62 36 56 66 63 95 37 34 137 174 156 129 224 90 15 59 75 96 103 105 91 158 139 164 156 78 169 163 191 115
33 19 36 26 20 

In [4]:
# helpers functions


def get_nodes_distance(a: int, b: int, problem: Problem) -> int:
    """Return distance between two nodes

    Args:
        a (int): from
        b (int): to
        problem (Problem): problem configuration

    Returns:
        int: distance
    """
    return problem.get_weight(a, b)


def get_problem_distance(solution: list[int], problem: Problem) -> int:
    """Get distance given a solution for the problem

    Args:
        solution (list[int]): list of nodes to travel
        problem (Problem): problem configuration

    Returns:
        int: total distance
    """

    conn = [*solution, solution[0]]

    return sum(
        map(
            lambda i: get_nodes_distance(conn[i], conn[i + 1], problem),
            range(len(conn) - 1),
        )
    )


def get_random_solution(problem: Problem) -> list[int]:
    """get a random solution for problem

    Args:
        problem (Problem): problem configuration

    Returns:
        list[int]: list of nodes to travel starting from 0
    """
    nodes = list(problem.get_nodes())
    random.shuffle(nodes)
    return nodes


solution = get_random_solution(problem)
print("Solution:", solution)

distance = get_problem_distance(solution, problem)
print("Distance:", distance)

Solution: [25, 23, 22, 26, 37, 29, 35, 17, 19, 34, 15, 38, 4, 1, 3, 33, 40, 39, 18, 21, 2, 41, 30, 36, 14, 12, 28, 5, 27, 8, 32, 16, 10, 7, 13, 11, 20, 9, 6, 31, 0, 24]
Distance: 4842


In [52]:
def get_best_in_population(
    population: list[list[int]], problem: Problem
) -> tuple[list[int], int]:
    """get best solution from population

    Args:
        population (list[list[int]]): population with solution
        problem (Problem): Problem config

    Returns:
        tuple[list[int], int]: best solution with calculated distance
    """
    best_sol, best_dist = [], math.inf

    for curr_sol in population[1:]:
        curr_dist = get_problem_distance(curr_sol, problem)
        if curr_dist < best_dist:
            best_sol = curr_sol
            best_dist = curr_dist

    return best_sol, best_dist


# Improvement 1: implement uniform cross of genes
def uniform_cross(parents: list, problem: Problem) -> list:
    """Uniform cross for genes parents

    Args:
        parents (list): tuple of par
        problem (Problem): problem config

    Returns:
        list: _description_
    """
    c1, c2 = [], []

    for i in range(len(parents[0])):
        swap_parents = random.random() > 0.5
        p1, p2 = parents[::-1] if swap_parents else parents

        c1.append(p1[i])
        c2.append(p2[i])

    c1 = ensure_valid_solution(c1, problem)
    c2 = ensure_valid_solution(c2, problem)

    return [c1, c2]


def ensure_valid_solution(solution: list[int], problem: Problem) -> list[list]:
    """Make sure that solution is valid after mutation, which means that all nodes from Problem have to be present

    Args:
        solution (list[int]): posible invalid solution
        problem (Problem): problem config

    Returns:
        list[list]: valid solution
    """
    # create iter object with the missing nodes in the solution
    missing_nodes = iter(set(problem.get_nodes()) - set(solution))
    seen = set()

    # iterate over the solution and replace duplicated with missing nodes
    return [
        next(missing_nodes, num) if num in seen else seen.add(num) or num
        for num in solution
    ]


# Improvement 3: Implement mutation using index popping strategy
def mutate_solution(solution: list[int]) -> list[int]:
    """Mutate solution by removing an element and inserting it at a random position.

    Args:
        solution (list[int]): solution for the problem

    Returns:
        list[int]: mutated solution
    """
    # copy solution to avoid params mutation
    mutation = solution[:]

    # get random positions
    i, j = random.sample(range(len(mutation)), 2)

    # remove element from position and insert it into the solution
    element = mutation.pop(i)
    mutation.insert(j, element)

    return mutation


def cross_population(
    population: list[list], mutation: float, problem: Problem
) -> list[list]:
    population_copy, population_final = copy.deepcopy(population), copy.deepcopy(
        population
    )

    # iterate over population until we don't have anymore couples
    while len(population_copy) > 1:
        # pick two random parents
        parents = random.sample(population_copy, 2)

        # remove parents from population
        for p in parents:
            population_copy.remove(p)

        # generate new children from parents
        children = uniform_cross(parents, problem)

        # mutate children depending on mutation
        mutated_children = map(
            lambda sol: mutate_solution(sol) if random.random() < mutation else sol,
            children,
        )

        # add them to final population
        population_final.extend([*children, *mutated_children])

    return population_final


def select_best_population(
    problem: Problem, population: list[list[int]], N: int, elitism: float
) -> list[list[int]]:
    """select best candidates for population based on elitism

    Args:
        problem (Problem): problem config
        population (list[list[int]]): all possible solutions
        N (int): amount of agents
        elitism (float): elitism value

    Returns:
        list[list[int]]: filtered solutions based on elitism
    """
    # sort populations by distance
    sorted_population = sorted(
        population,
        key=lambda sol: get_problem_distance(sol, problem),
    )

    # split into best solutions and rest
    n_amount = int(N * elitism)
    best_sols, rest_sols = (sorted_population[:n_amount], sorted_population[n_amount:])

    # return best solutions and the rest randomized
    return best_sols + random.sample(rest_sols, int(N * (1 - elitism)))


def genetic_algorithm(
    mutation_range: tuple[int, int],
    elitism_range: tuple[int, int],
    problem: Problem,
    N=100,
    gen=100,
) -> tuple[list[int], int]:
    """Genetic algorithm for TSP

    Args:
        mutation_range (tuple[int, int]): range of mutation to use during the generations
        elitism_range (tuple[int, int]): range for elitism to use during the generations
        problem (Problem): problem config
        N (int, optional): amount of agents. Defaults to 100.
        gen (int, optional): amount of generations. Defaults to 100.

    Returns:
        tuple[list[int], int]: best solution along with its distance
    """
    # Generate population with random solution with the length of agents
    population = [get_random_solution(problem) for _ in range(N)]

    # Improvements 2: use a adaptive mutation and elitism rates
    elitism_arr = np.linspace(*elitism_range, gen)
    mutation_arr = np.linspace(*mutation_range[::-1], gen)

    for i in range(gen):
        mutation, elitism = mutation_arr[i], elitism_arr[i]

        # cross parents to get new children + possible mutation
        population = cross_population(population, mutation, problem)

        # use elitism to select population
        population = select_best_population(problem, population, N, elitism)

        # print progress
        (best_sol, best_dist) = get_best_in_population(population, problem)
        print(f"Generacion #{i + 1}")
        print(f"La mejor solución es: {best_sol}")
        print(f"Distancia: {best_dist}")
        print(f"Elitismo: {elitism} | Mutación: {mutation}")
        print("-------")

    # return best solution + distance
    return (best_sol, best_dist)


genetic_algorithm(
    mutation_range=(0.5, 0.8),
    elitism_range=(0.45, 0.9),
    problem=problem,
    N=300,
    gen=300,
)

Generacion #1
La mejor solución es: [3, 27, 2, 20, 6, 9, 24, 7, 41, 25, 18, 4, 17, 31, 37, 35, 32, 19, 38, 34, 14, 36, 28, 40, 8, 21, 23, 33, 11, 13, 1, 16, 5, 39, 22, 26, 15, 0, 10, 12, 29, 30]
Distancia: 4030
Elitismo: 0.45 | Mutación: 0.8
-------
Generacion #2
La mejor solución es: [3, 9, 30, 11, 6, 20, 5, 10, 0, 2, 4, 17, 36, 1, 29, 34, 32, 14, 35, 33, 41, 37, 31, 38, 19, 12, 8, 39, 21, 40, 25, 16, 26, 18, 23, 24, 22, 15, 13, 27, 7, 28]
Distancia: 3948
Elitismo: 0.45150501672240806 | Mutación: 0.7989966555183947
-------
Generacion #3
La mejor solución es: [19, 12, 28, 18, 25, 34, 10, 4, 5, 7, 8, 38, 20, 16, 15, 37, 13, 0, 35, 1, 32, 17, 26, 33, 36, 6, 23, 39, 40, 9, 41, 24, 11, 3, 27, 29, 2, 14, 31, 21, 22, 30]
Distancia: 3894
Elitismo: 0.45301003344481605 | Mutación: 0.7979933110367894
-------
Generacion #4
La mejor solución es: [20, 38, 24, 41, 19, 6, 0, 32, 28, 34, 33, 35, 3, 8, 18, 4, 36, 40, 21, 9, 12, 5, 13, 1, 22, 27, 17, 2, 37, 7, 30, 15, 39, 23, 10, 11, 14, 16, 25, 26, 29,

([3,
  4,
  6,
  1,
  0,
  32,
  34,
  20,
  7,
  37,
  17,
  31,
  36,
  35,
  33,
  38,
  22,
  39,
  24,
  40,
  21,
  9,
  8,
  10,
  25,
  11,
  12,
  13,
  19,
  16,
  15,
  14,
  5,
  26,
  18,
  41,
  23,
  30,
  29,
  28,
  27,
  2],
 1572)

Mejor solución obtenida con el algoritmo de arriba:

```python
Generacion #300
La mejor solución es: [3, 4, 6, 1, 0, 32, 34, 20, 7, 37, 17, 31, 36, 35, 33, 38, 22, 39, 24, 40, 21, 9, 8, 10, 25, 11, 12, 13, 19, 16, 15, 14, 5, 26, 18, 41, 23, 30, 29, 28, 27, 2]
Distancia: 1572
Elitismo: 0.9 | Mutación: 0.5
```

Utilizando los siguientes paramétros:

```python
genetic_algorithm(
    mutation_range=(0.5, 0.8),
    elitism_range=(0.45, 0.9),
    problem=problem,
    N=300,
    gen=300,
)
```
